In [ ]:
import numpy as np
%load_ext Cython

# Advanced Python Programming – Lab 8
In this Lab, we'll use iteration over arrays to implement a more efficient way to compute the squared sum of a matrix using Cython. This example is extracted from the [Numpy Documentation](https://docs.scipy.org/doc/numpy/reference/arrays.nditer.html).

# Questions?

Difference between compile-time type and run-time type?
For every type in the numpy module there's a corresponding compile-time type with a **\_t-suffix**.

## Warm-up
Check out how [numpy iterators](https://docs.scipy.org/doc/numpy/reference/generated/numpy.nditer.html#numpy.nditer) work.

In [2]:
a = np.arange(6).reshape(2, 3)

In [3]:
a

array([[0, 1, 2],
       [3, 4, 5]])

In [4]:
for x in np.nditer(a):
    print(x)

0
1
2
3
4
5


Notice that by deafault iterators traverse array elements in the same order they are stored in memory for efficiency.

In [5]:
for x in np.nditer(a.T):
    print(x)

0
1
2
3
4
5


## 1) How would you compute the square sum of a matrix using built-in Numpy functions? Measure the time it takes to compute for a BIG matrix

In [6]:
b = np.arange(10000000).reshape(10000, 1000)

In [7]:
%timeit np.sum(np.square(a))
# or np.sum(a*a)
# or np.sum(a**2)

The slowest run took 4.80 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 25.1 µs per loop


In [8]:
%timeit np.sum(np.square(b))

10 loops, best of 3: 66.6 ms per loop


## Now let's build our own function to calculate the square sum using Cython and beat that time!
... but first, we need to learn some other concepts about array iteration

### Modifying Array Values
What happens if you execute the following code and why?

In [9]:
for x in np.nditer(a, op_flags=['readwrite']):
    x = 2 * x
    print(x)

0
2
4
6
8
10


In [10]:
a

array([[0, 1, 2],
       [3, 4, 5]])

In [11]:
for x in np.nditer(a, op_flags=['readwrite']):
    x[...] = 2 * x
    print(x)

0
2
4
6
8
10


In [12]:
a

array([[ 0,  2,  4],
       [ 6,  8, 10]])

### Using an External Loop
The **external_loop** flag option lets you iterate the elements by chunks instead of one by one.

Check out the **buffered** option too, what is its purpose?

In [13]:
for x in np.nditer(a, flags=['external_loop']):
     print(x)

[ 0  2  4  6  8 10]


## 2) Given the following code to perform a squared sum of elements in a matrix with respect to specified axis, Cython-ize it so that it is more efficient. 

Measure the difference with respect to the answer in question 1 and to the pure Python + Numpy implementation.

In [14]:
def axis_to_axeslist(axis, ndim):
    if axis is None:
        return [-1] * ndim
    else:
        if type(axis) is not tuple:
            axis = (axis,)
        axeslist = [1] * ndim
        for i in axis:
            axeslist[i] = -1
        ax = 0
        for i in range(ndim):
            if axeslist[i] != -1:
                axeslist[i] = ax
                ax += 1
        return axeslist

def sum_squares_py(arr, axis=None, out=None):
    axeslist = axis_to_axeslist(axis, arr.ndim)
    it = np.nditer([arr, out], flags=['reduce_ok', 'external_loop',
                                      'buffered', 'delay_bufalloc'],
                op_flags=[['readonly'], ['readwrite', 'allocate']],
                op_axes=[None, axeslist],
                op_dtypes=['float64', 'float64'])
    it.operands[1][...] = 0
    it.reset()
    for x, y in it:
        y[...] += x*x
    return it.operands[1]


# ... syntax in python
#    http://stackoverflow.com/questions/772124/what-does-the-python-ellipsis-object-do
#    http://stackoverflow.com/questions/42190783/what-does-three-dots-in-python-mean-when-indexing-what-looks-like-a-number

In [15]:
c = np.arange(6).reshape(2,3)

In [16]:
%timeit sum_squares_py(c)

10000 loops, best of 3: 62.6 µs per loop


In [17]:
sum_squares_py(c, axis=-1)

array([  5.,  50.])

In [18]:
%reload_ext Cython

In [22]:
%%cython  # this magic command must be the first line in a cell

import numpy as np
cimport numpy as np
import cython

# refer to the cython array iterate part
# https://docs.scipy.org/doc/numpy/reference/arrays.nditer.html

def axis_to_axeslist(axis, ndim):
    if axis is None:
        return [-1] * ndim
    else:
        if type(axis) is not tuple:
            axis = (axis,)
        axeslist = [1] * ndim
        for i in axis:
            axeslist[i] = -1
        ax = 0
        for i in range(ndim):
            if axeslist[i] != -1:
                axeslist[i] = ax
                ax += 1
        return axeslist
    
@cython.wraparound(False)
@cython.boundscheck(False)
cdef sum_squares_py(arr, axis=None, out=None):
    axeslist = axis_to_axeslist(axis, arr.ndim)
    it = np.nditer([arr, out], flags=['reduce_ok', 'external_loop',
                                      'buffered', 'delay_bufalloc'],
                op_flags=[['readonly'], ['readwrite', 'allocate']],
                op_axes=[None, axeslist],
                op_dtypes=['float64', 'float64'])
    it.operands[1][...] = 0
    it.reset()
    cdef double x
    cdef double y
    for x, y in it:
        y[...] += x*x
    return it.operands[1]

UsageError: unrecognized arguments: # this magic command must be the first line in a cell


In [ ]:
%timeit sum_squares_py(c)

In [ ]:
%timeit sum_squares_py(b)

In [27]:
print(np.arange.__doc__)


arange([start,] stop[, step,], dtype=None)

    Return evenly spaced values within a given interval.

    Values are generated within the half-open interval ``[start, stop)``
    (in other words, the interval including `start` but excluding `stop`).
    For integer arguments the function is equivalent to the Python built-in
    `range <http://docs.python.org/lib/built-in-funcs.html>`_ function,
    but returns an ndarray rather than a list.

    When using a non-integer step, such as 0.1, the results will often not
    be consistent.  It is better to use ``linspace`` for these cases.

    Parameters
    ----------
    start : number, optional
        Start of interval.  The interval includes this value.  The default
        start value is 0.
    stop : number
        End of interval.  The interval does not include this value, except
        in some cases where `step` is not an integer and floating point
        round-off affects the length of `out`.
    step : number, optional
        

In [37]:
a = np.arange(2,5)
print(a**2)
print(np.sum(a**2)**-1.5)
print((a[0]*a[0] + a[1]*a[1] + a[2]*a[2]) ** -1.5)
a.tolist() + [1,2,3]

[ 4  9 16]
0.00640328752335
0.00640328752335


[2, 3, 4, 1, 2, 3]